# Calculating and visualizing the climatology from monthly model output
#### Important: This needs to run with the "calc" python environment
- First import numpy and xarray libraries

In [ ]:
import numpy as np
import netCDF4
import xarray as xr
import matplotlib.pyplot as plt

- Next, we need to read in the data. Let's pick SST (tos) from GFDL-ESM4
- Since we can't remember the exact filename, we use "ls" command to find the file and save it as a text file

In [ ]:
! ls /global/cfs/cdirs/m3920/dataset/cmip6/GFDL-ESM4/tos*_gr_*.nc > gfdltos.txt
! cat gfdltos.txt

- First read in the text file that contains the name of the file
- Use it to drive xarray.open_dataset
- See the graphic interface to explore metadata and attributes

In [ ]:
fid=open('gfdltos.txt','r')
fn=fid.read().splitlines()
ds=xr.open_dataset(fn[0])
ds

- Calculate the monthly climatology from 1950 to 2010
- To do so, we first select the correct time period
- Notice that the number of time slice is reduced to 732 (= 12 x 61)


In [ ]:
sst_select = ds.tos.sel(time=slice("1950-01-01","2010-12-31"))
np.shape(sst_select)

- Group the data by month and take time average
- Make sure that there will be 12 time slices as the output
- Use %%time to measure performance

In [ ]:
%%time
sst_monclim = sst_select.groupby('time.month').mean(dim='time')
np.shape(sst_monclim)

- Similarly, we can calculate seasonal climatology as follows
- One bug is that the time axis became alphabetical, thus needs to be corrected

In [ ]:
%%time
sst_seaclim = sst_select.groupby('time.season').mean(dim='time')
sst_seaclim

In [ ]:
sst_seaclim=sst_seaclim.reindex(season=['DJF','MAM','JJA','SON'])

- To conclude, let's plot the seasonal SST climatology
- Explore xarray's .plot function

In [ ]:
sst_seaclim.sel(season='DJF').plot(robust=True)

- Faceting allows multiple panels from the 3D array

In [ ]:
sst_seaclim.plot(col="season",col_wrap=2,robust=True)

- Subtract annual mean to show seasonal changes
- Print a PDF file by uncommenting the last line

In [ ]:
ssta_seaclim=sst_seaclim - sst_seaclim.mean(dim='season')
colorbar_kwargs = {
    "orientation": "vertical",
    "label": "seasonal SST departure from annual mean",
    "pad": 0.07,}
ssta_seaclim.plot(col="season",col_wrap=2,robust=True, figsize=(7,5), cbar_kwargs=colorbar_kwargs)
#plt.savefig('seasonal_sst.pdf',bbox_inches="tight")